# Background

| <p align="left">Parameters</p>          | <p align="left">Estimate</p>                      |
|-----------------------------------------|---------------------------------------------------|
| <p align="left">$L^T \beta$</p>         | <p align="left">$L^T \widehat{\beta}$</p>         |
| <p align="left">$\exp{(L^T \beta)}$</p> | <p align="left">$\exp{(L^T \widehat{\beta)}}$</p> |
| <p align="left">$\pi = \frac{1}{1 + \exp{(L^T \beta)}}$</p> | <p align="left">$\widehat{\pi} = \frac{1}{1 + \exp{(L^T \widehat{\beta})}}$</p> |

| <p align="left">Parameters</p>          | <p align="left">Estimate</p>                      |
|-----------------------------------------|---------------------------------------------------|
| <p align="left">95% CI for true $L^T \beta$</p>         | <p align="left">$L^T \widehat{\beta} \pm 1.96 \sqrt{V(L^T \widehat{\beta})}$</p>         |
| <p align="left">95% CI for true $\exp{(L^T \beta)}$</p> | <p align="left">$exp \Big(L^T \widehat{\beta} \pm 1.96 \sqrt{V(L^T \widehat{\beta})}\Big)$</p> |
| <p align="left">95% CI for true $\pi$</p> | <p align="left">$\frac{1}{1 + exp \Big(L^T \widehat{\beta} \pm 1.96 \sqrt{V(L^T \widehat{\beta})}\Big)}$</p> |

# Set environment

In [1]:
library(tidyverse)
library(IRdisplay)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.0.0     ✔ purrr   0.2.5
✔ tibble  1.4.2     ✔ dplyr   0.7.6
✔ tidyr   0.8.1     ✔ stringr 1.3.1
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


# Import data

In [2]:
source("./data/table_7_5_embryogenic_athers.R")
embryogenic_anthers

y,n,storage,centrifuge,newstor,force
55,102,1,40,0,3.688879
52,99,1,150,0,5.010635
57,108,1,350,0,5.857933
55,76,2,40,1,3.688879
50,81,2,150,1,5.010635
50,90,2,350,1,5.857933


# Fit model

$logit(p) = \beta_1 + \beta_2 \text{newstor}$

Note: newstor: 
- 1 = treatment 
- 0 = control

In [3]:
fit = glm(cbind(y, n-y) ~ newstor, data = embryogenic_anthers, family = binomial(link = "logit"))
summary(fit)


Call:
glm(formula = cbind(y, n - y) ~ newstor, family = binomial(link = "logit"), 
    data = embryogenic_anthers)

Deviance Residuals: 
       1         2         3         4         5         6  
 0.17150  -0.10947  -0.06177   1.77208  -0.19040  -1.39686  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)  
(Intercept)   0.1231     0.1140   1.080   0.2801  
newstor       0.3985     0.1741   2.289   0.0221 *
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 10.452  on 5  degrees of freedom
Residual deviance:  5.173  on 4  degrees of freedom
AIC: 38.741

Number of Fisher Scoring iterations: 3


# Odds ratio<sub>*pattern1 vs pattern2*</sub>

- Pattern 1: treatment
- Pattern 2: control

$\Rightarrow$ OR<sub>*pattern1 vs pattern2*</sub> 

$= \frac{exp\Big( \beta_1 + \beta_2 x 1 \Big)}{exp\Big( \beta_1 + \beta_2 x 0 \Big)} = exp\{\beta_2\}$

$\Rightarrow$ Contrast matrix (coefficients of a linear combination of parameters)

$L = \begin{bmatrix} 0 \\ 1 \end{bmatrix}$

$L^T \beta = \begin{bmatrix} 0 & 1 \end{bmatrix} \begin{bmatrix} \beta_1 \\ \beta_2 \end{bmatrix} = \beta_2$

$V\Big( L^T \beta\Big) = 
     \begin{bmatrix} 0 & 1 \end{bmatrix} 
     \begin{bmatrix} 
         V(\beta_1)            & Cov(\beta_1, \beta_2) \\
         Cov(\beta_1, \beta_2) & V(\beta_2)            \\
     \end{bmatrix} 
     \begin{bmatrix} 0 \\ 1 \end{bmatrix} = 
     V(\beta_2)$

In [4]:
L = cbind(c(0, 1))
print(L)

     [,1]
[1,]    0
[2,]    1


## estimate $L^T \beta$ and $exp \big( L^T \beta \big)$

$\text{logit(p)} = L^T \beta$

In [5]:
t(L) %*% fit$coef

0.3985039


estimated OR $= exp(L^T \beta)$

In [6]:
t(L) %*% fit$coef %>% exp

1.489594


## estimate var($L^T \beta$)

$cov \big( \beta \big)$

In [7]:
covb = summary(fit)$cov.unscaled
display(covb)

,(Intercept),newstor
(Intercept),0.01299411,-0.01299411
newstor,-0.01299411,0.03031520


$L^T cov \big( \beta \big) L$

In [8]:
t(L) %*% covb %*% L

0.0303152


standard deviation

In [9]:
t(L) %*% covb %*% L %>% sqrt

0.1741126


## 95% CI for true $L^T \beta$ and true OR

95% CI for true $L^T \beta$

In [10]:
Lb    = t(L) %*% fit$coef   %>% as.vector
varLb = t(L) %*% covb %*% L %>% as.vector
 
CILb        = Lb + c(-1, 1) * qnorm(1 - 0.05 / 2) * sqrt(varLb)
names(CILb) = c("lower", "upper")

display(CILb)

lower      upper 
0.05724941 0.73975830

95% CI for true OR

In [11]:
exp(CILb)

lower    upper 
1.058920 2.095429

# Another contrast matrix

Covariate Pattern $L = \begin{bmatrix} 1 \\ 1 \end{bmatrix}$  

$\Rightarrow \beta_1 + \beta_2$

which is the odds of treatment   

$\Rightarrow \text{logit(p)} = \beta_1 + \beta_2 newstor$

Therefore, we can **estimate predicted probability of success (and CI) for the pattern**

In [12]:
L = cbind(c(1, 1))
print(L)

     [,1]
[1,]    1
[2,]    1


## estimate $L^T \beta$ and $exp \big( L^T \beta \big)$

In [15]:
Lb = t(L) %*% fit$coef %>% as.vector
print(Lb)

[1] 0.5216365


In [16]:
Lb %>% exp

[1] 1.684783

## estimate var($L^T \beta$)

In [19]:
covb = summary(fit)$cov.unscaled
display(covb)

,(Intercept),newstor
(Intercept),0.01299411,-0.01299411
newstor,-0.01299411,0.03031520


In [20]:
varLb = t(L) %*% covb %*% L %>% as.vector
display(varLb)

[1] 0.01732109

## 95% CI for true $L^T \beta$ and true probability

95% CI for true $L^T \beta$

In [21]:
CILb = Lb + c(-1, 1) * qnorm(1 - 0.05 / 2) * sqrt(varLb)
names(CILb) = c("lower", "upper")

display(CILb)

lower     upper 
0.2636865 0.7795866

95% CI for true odds

In [23]:
display(CILb %>% exp)

lower    upper 
1.301720 2.180571

95% CI for success probability in treatment

In [29]:
f = function(x){ return(1 / x) }
display(CILb %>% `-` %>% exp %>% `+`(., 1) %>% f)

lower     upper 
0.5655423 0.6855910

# add interaction term

$logit(p) = \beta_1 + \beta_2 newstor + \beta_3 newstor * force$

- pattern 1: treatment, force = 5  
- pattern 2: control, force = 4

$OR_{pattern 1 vs pattern 2}$

$= \frac{
    exp \big( \beta_1 + \beta_2 * 1 + \beta_3 * 5 + \beta_4 * 1 * 5 \big)
}{
    exp \big( \beta_1 + \beta_2 * 0 + \beta_3 * 4 + \beta_4 * 0 * 4 \big)
}$

$= exp \big( \beta_2 + \beta_3 + 5 \beta_4 \big)$

contrast matrix

$L^T = \begin{bmatrix} 0 & 1 & 1 & 5 \end{bmatrix}$

In [30]:
L = cbind(c(0, 1, 1, 5))
print(L)

     [,1]
[1,]    0
[2,]    1
[3,]    1
[4,]    5


## fit model

In [31]:
embryogenic_anthers

y,n,storage,centrifuge,newstor,force
55,102,1,40,0,3.688879
52,99,1,150,0,5.010635
57,108,1,350,0,5.857933
55,76,2,40,1,3.688879
50,81,2,150,1,5.010635
50,90,2,350,1,5.857933


In [34]:
fit = glm(cbind(y, n - y) ~ newstor + force + newstor * force, data = embryogenic_anthers, binomial(link = "logit"))
print(fit)


Call:  glm(formula = cbind(y, n - y) ~ newstor + force + newstor * force, 
    family = binomial(link = "logit"), data = embryogenic_anthers)

Coefficients:
  (Intercept)        newstor          force  newstor:force  
      0.23389        1.97711       -0.02274       -0.31862  

Degrees of Freedom: 5 Total (i.e. Null);  2 Residual
Null Deviance:	    10.45 
Residual Deviance: 0.02773 	AIC: 37.6


## Estimation

$L^T \beta$

In [50]:
Lb   = t(L) %*% fit$coef %>% as.vector
display(Lb)

[1] 0.3612751

$exp \big( L^T \beta \big)$

In [52]:
display(exp(Lb))

[1] 1.435158

$cov \big( \beta \big)$

In [43]:
covb = summary(fit)$cov.unscaled
display(covb)

,(Intercept),newstor,force,newstor:force
(Intercept),0.39487339,-0.39487339,-0.07838818,0.07838818
newstor,-0.39487339,0.99605180,0.07838818,-0.19539027
force,-0.07838818,0.07838818,0.01609076,-0.01609076
newstor:force,0.07838818,-0.19539027,-0.01609076,0.03955189


$var \big( L^T \beta \big)$

In [51]:
varLb = t(L) %*% covb %*% L %>% as.vector
display(sqrt(varLb))

[1] 0.2071373

95% CI for Lb

In [46]:
CILb  = Lb + c(-1, 1) * qnorm(1 - 0.05 / 2) * sqrt(varLb) 
display(CILb)

[1] -0.04470659  0.76725671

95% CI for true OR

In [48]:
display(CILb %>% exp)

[1] 0.956278 2.153850